# MMVCの学習

ver.2022/10/30

MMVCで利用するVITSの学習を行います。

​

In [ ]:
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#学習に必要なパッケージ群
!apt-get install espeak
!pip install -r requirements.txt
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
#@title ## 2 リポジトリの準備
#@markdown リポジトリの準備を行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。

#@markdown 設定が完了したら、次へを押してください。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

#---関数終わり---

def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  #global directory_input #グローバル指定
  #global branch_input #グローバル指定
  global path_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  path_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer_main', placeholder='/content/drive/MyDrive/MMVC_Trainer_main', description='Path:', disabled=False)
  #directory_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer', placeholder='/content/drive/MyDrive/MMVC_Trainer', description='Directory:', disabled=False)
  #branch_input = widgets.Text(value='main', placeholder='main', description='Branch:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, path_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  #global directory #グローバル指定
  #directory = directory_input.value
  global platform #グローバル指定
  platform = platform_input.value
  #global branch #グローバル指定
  #branch = branch_input.value
  global path #グローバル指定
  #path = directory + "/MMVC_Trainer_" + branch
  path = path_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    %cd $path
  elif platform == "Local":
    %cd $path

#実行
first_view()

​

In [ ]:
#@title ## 3 実行の準備
#@markdown 実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

print("log: パッケージのインストールを開始します。")
!apt-get install espeak
!pip install -r requirements.txt
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
#@title ## 4 GPUの確認
#@markdown GPUの確認を行います。
#@markdown 割り当てられたGPUのメモリーを確認し、必要に応じてconfigファイルの"batch_size"を変更してください。 

#@markdown よく分からない場合、変更不要です。(学習においてメモリの不足によりエラーとなる場合、変更が必要となります。詳細についてはDiscordでお問い合わせください)
!nvidia-smi

​

In [ ]:
#@title ## 5 tensorboardの起動
#@markdown とりあえず実行しておいてください。

#Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs --port 5000

​

In [ ]:
#@title ## 6 学習を実行する
#@markdown 学習を実行します。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown New / Resume
#@markdown 新規に学習を開始する場合は、Newを選択し、**以下の-c, -m, -fg, -fdを全て設定してください。**
#@markdown 学習を再開する場合は、Resumeを選択し、以下の-c, -mを設定してください。-mで指定したディレクトリの最新のモデルから学習を再開します。

new_or_resume = "new"

#@markdown -c：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。

config_pass = "configs/train_config.json"

#@markdown -m：modelの保存先ディレクトリ  
#@markdown **ディレクトリ名を直接指名してください。** 

model_save_dic = "20220306_24000"

#@markdown -fg：(新規学習時のみ) Fine tuningのベースとなるG_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。

fine_model_g = "fine_model/G_180000.pth"

#@markdown -fd：(新規学習時のみ) Fine tuningのベースとなるD_xxxx.pth のpathを指定してください。  
#@markdown よく分からない場合は、変更不要です。

fine_model_d = "fine_model/D_180000.pth"

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。

def first_view():
  #widgetsの構成
  global new_or_resume_input #グローバル指定
  global config_pass_input #グローバル指定
  global model_save_dic_input #グローバル指定
  global fine_model_g_input #グローバル指定
  global fine_model_d_input #グローバル指定
  new_or_resume_input = widgets.Dropdown(options=["New", "Resume"], value="New", description='new/resume:', disabled=False)
  config_pass_input = widgets.Text(value='configs/train_config.json', placeholder='configs/train_config.json', description='config_pass:', disabled=False)
  model_save_dic_input = widgets.Text(value='20220306_24000', placeholder='20220306_24000', description='model_save_dic_input:', disabled=False)
  fine_model_g_input = widgets.Text(value='fine_model/G_180000.pth', placeholder='fine_model/G_180000.pth', description='fine_model_g:', disabled=False)
  fine_model_d_input = widgets.Text(value='fine_model/D_180000.pth', placeholder='fine_model/D_180000.pth', description='fine_model_d:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(new_or_resume_input, config_pass_input, model_save_dic_input, fine_model_g_input, fine_model_d_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global new_or_resume #グローバル指定
  new_or_resume = new_or_resume_input.value
  global config_pass #グローバル指定
  config_pass = config_pass_input.value
  global model_save_dic #グローバル指定
  model_save_dic = model_save_dic_input.value
  global fine_model_g #グローバル指定
  fine_model_g = fine_model_g_input.value
  global fine_model_d #グローバル指定
  fine_model_d = fine_model_d_input.value
  if New_or_Resume == "New":
    !python train_ms.py -c $config_pass -m $model_save_dic -fg $fine_model_g -fd $fine_model_d
  elif New_or_Resume == "Resume":
    !python train_ms.py -c $config_pass -m $model_save_dic
  

#実行
first_view()